# Engineering Assignment 2: Bridge Load Analysis
**Brian Capellan-Santos** 


## Introduction

The purpose of this analysis is to evaluate how different loads affect a simple bridge structure.

This report calculates:
- Total span of bridge in meters
- Force applied
- Distance from left support to load in meters
- Position along bridge to calculate moment

The goal is to determine whether each bridge design is structurally adequate for the given load.

## Function 1: calculate_single_moment

This function calculates the internal bending moment at a specific location on a simply supported bridge under a single point load.

In [1]:
def calculate_single_moment(bridge_length, load, load_position, x):    
    b = bridge_length - load_position
    
    if x <= load_position:
        return (load * b * x) / bridge_length
    else:
        return (load * load_position * (bridge_length - x)) / bridge_length

## Function 2: calculate_deflection

This function calculates vertical deflection at a given point along the bridge using beam deflection equations.

In [2]:
def calculate_deflection(bridge_length, load, load_position, x, EI):    
    L = bridge_length
    P = load
    a = load_position
    b = L - a
    
    if x <= a:
        delta = (P * b * x) / (6 * L * EI) * (L**2 - b**2 - x**2)
    else:
        delta = (P * a) / (6 * L * EI) * ((L - x) * (2 * L * x - x**2 - a**2))
    
    return delta

## Function 3: create_analysis_summary

This function:
- Finds maximum bending moment
- Finds maximum deflection
- Calculates deflection ratio
- Checks span/360 requirement

In [4]:
import numpy as np

def create_analysis_summary(bridge_specs, load_specs):
    L = bridge_specs['length']
    EI = bridge_specs['EI']
    P = load_specs['magnitude']
    a = load_specs['position']
    
    # Maximum moment occurs at load position
    max_moment = calculate_single_moment(L, P, a, a)
    max_moment_location = a
    
    # This creates 101 evenly spaced points along the bridge.
    # Because we want to evaluate deflection across the entire beam to find the maximum.
    x_values = np.linspace(0, L, 101)
    deflections = [calculate_deflection(L, P, a, x, EI) for x in x_values]
    
    max_deflection = max(deflections)
    # finds where the biggest deflection occurs.
    max_deflection_location = x_values[np.argmax(deflections)]
    
    # Convert to mm standard
    max_deflection_mm = max_deflection * 1000
    
    deflection_ratio = L / max_deflection if max_deflection != 0 else float('inf')
    
    # It’s checking:
    # Does it sag too much?
    # Would users feel uncomfortable?
    # Would cracks form?
    allowable_deflection = L / 360
    status = "PASS" if max_deflection <= allowable_deflection else "FAIL"
    
    return {
        'max_moment': max_moment,
        'max_moment_location': max_moment_location,
        'max_deflection_mm': max_deflection_mm,
        'max_deflection_location': max_deflection_location,
        'deflection_ratio': deflection_ratio,
        'status': status
    }

## Scenario 1: Pedestrian Footbridge

In [5]:
bridge1 = {'length': 10, 'EI': 5000, 'name': 'Timber Footbridge'}
load1 = {'magnitude': 0.8, 'position': 5, 'type': 'Pedestrian'}

result1 = create_analysis_summary(bridge1, load1)

result1

{'max_moment': 2.0,
 'max_moment_location': 5,
 'max_deflection_mm': 3.3333333333333335,
 'max_deflection_location': 5.0,
 'deflection_ratio': 3000.0,
 'status': 'PASS'}

## Scenario 2: Residential Street Bridge

In [6]:
bridge2 = {'length': 15, 'EI': 25000, 'name': 'Concrete Bridge'}
load2 = {'magnitude': 15, 'position': 6, 'type': 'Vehicle'}

result2 = create_analysis_summary(bridge2, load2)
result2

{'max_moment': 54.0,
 'max_moment_location': 6,
 'max_deflection_mm': 40.003605000000015,
 'max_deflection_location': 7.05,
 'deflection_ratio': 374.9662061706688,
 'status': 'PASS'}

## Scenario 3: Highway Bridge

In [7]:
bridge3 = {'length': 25, 'EI': 150000, 'name': 'Highway Bridge'}
load3 = {'magnitude': 50, 'position': 10, 'type': 'Truck'}

result3 = create_analysis_summary(bridge3, load3)
result3

{'max_moment': 300.0,
 'max_moment_location': 10,
 'max_deflection_mm': 102.88993055555557,
 'max_deflection_location': 11.75,
 'deflection_ratio': 242.9781015985934,
 'status': 'FAIL'}

## Conclusion

This analysis evaluated bridge performance under pedestrian, vehicle, and truck loads.

Results show:
- Maximum moment occurs at load position.
- Deflection increases with load magnitude and decreases with higher EI.

Analysis like this ensures safety and prevents failure under real loading conditions.


## Question 1

In [9]:
L = 15
P = 15

for pos in [4, 7.5, 11]:
    moment = calculate_single_moment(L, P, pos, pos)
    print(f"Load at {pos} m -> Moment = {moment} kN-m")

Load at 4 m -> Moment = 44.0 kN-m
Load at 7.5 m -> Moment = 56.25 kN-m
Load at 11 m -> Moment = 44.0 kN-m


The moment is maximum when the load is near midspan (7.5m). 
Moment is largest near the center because the lever arm distance is maximized.

## Question 2

Scenario 3 has the largest absolute deflection due to highest load.

Worst deflection ratio = smallest L/δ value.

## Question 3

In [10]:
for bridge in [bridge1, bridge2, bridge3]:
    allowable = bridge['length'] / 360
    print(f"{bridge['name']} allowable deflection:", allowable, "m")

Timber Footbridge allowable deflection: 0.027777777777777776 m
Concrete Bridge allowable deflection: 0.041666666666666664 m
Highway Bridge allowable deflection: 0.06944444444444445 m


## Question 4

In [11]:
load3_center = {'magnitude': 50, 'position': 12.5, 'type': 'Truck'}
center_result = create_analysis_summary(bridge3, load3_center)
center_result

{'max_moment': 312.5,
 'max_moment_location': 12.5,
 'max_deflection_mm': 108.50694444444444,
 'max_deflection_location': 12.5,
 'deflection_ratio': 230.4,
 'status': 'FAIL'}

Deflection increases when the truck is at the exact center.
Maximum bending and deflection occur at midspan.

## Question 5

In [12]:
# Double load for Scenario 1
double_load = 1.6
moment_double = calculate_single_moment(10, double_load, 5, 5)
moment_double

4.0

Doubling the load doubles the bending moment.

To keep deflection within code, EI would also need to double,
since deflection is inversely proportional to EI.